In [1]:
import sys
sys.path.append('../utils')
import fvdb.nn as fvnn
import torch 
import igl
from diffusion_tensor import *
import mesh_tools as mt
import matplotlib.pyplot as plt
from fvdb_utils import show_vdb_marching_cubes, grid_to_VDB
from tqdm import tqdm


### (OLD) Init

In [80]:
ind=2
tens = torch.load('../../output/house/gen_0_4.pt', weights_only=False)
disp_d = DiffusionTensor(tens.grid[ind], tens.feature[ind]).remove_mask()
disp_d.get_global().colored_PC(.07)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.8114528…

### (OLD) Test direct MC

In [4]:
train_grid = disp_d
train_grid.feature.jdata[:, 3:6] = torch.clip(train_grid.feature.jdata[:, 3:6], -.5, .5)

In [5]:
dual_grid = train_grid.grid.dual_grid()
dual_centers = dual_grid.grid_to_world(dual_grid.ijk.float())
normals, vstars, _, mask = DiffusionTensor.get_feature_data(train_grid.get_global().jdata)
# splat vstars + normals on dual grid
print(vstars.shape)
features = torch.cat((normals, (vstars*normals).sum(-1, True), torch.ones_like(vstars[:, :1])), -1)
new_feature = dual_grid.splat_trilinear(train_grid.grid.jagged_like(vstars), train_grid.grid.jagged_like(features))

nfdata = new_feature.jdata[:, :-1]/new_feature.jdata[:, -1:]
dual_sign = (dual_centers.jdata*nfdata[:,:3]).sum(-1, True)-nfdata[:,3:]
dual_sign = 2.*(dual_sign>0)-1
dual_tens = fvnn.VDBTensor(dual_grid, dual_grid.jagged_like(dual_sign[:, None]))

torch.Size([118504, 3])


In [6]:
dual_data = dual_tens.to_dense().cpu().detach().numpy()
primal_data = train_grid.to_dense().cpu().detach().numpy()

mt.plotSlice(dual_data[0, ..., 0], .01)

interactive(children=(IntSlider(value=114, description='s', max=228), Output()), _dom_classes=('widget-interac…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [7]:
show_vdb_marching_cubes(dual_tens)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### (OLD) Robust hierarchy

In [4]:
def get_pool(self, k_s=2):
    '''make sure it's global first'''
    feature, grid = self.grid.avg_pool(k_s, self.feature, k_s)
    feature.jdata /= feature.jdata[:, -1:].clone()
    return  DiffusionTensor(grid, feature)


In [9]:
global_fine_grid = disp_d.get_global()

first_ks = 128
primal_data = get_pool(global_fine_grid, first_ks)

if not primal_data.to_dense()[..., -1].sum()==8:
    print('WARNING: ASSUMPTION BROKEN')
occupancy_tensor = fvnn.VDBTensor(primal_data.grid, primal_data.grid.jagged_like(torch.zeros(8, device='cuda')[:, None]))
# -1: INSIDE, 0: ACTIVE, 1: OUTSIDE

In [10]:
primal_data.grid.ijk.jdata

tensor([[-1, -1, -1],
        [-1, -1,  0],
        [-1,  0, -1],
        [-1,  0,  0],
        [ 0, -1, -1],
        [ 0, -1,  0],
        [ 0,  0, -1],
        [ 0,  0,  0]], device='cuda:0', dtype=torch.int32)

In [11]:
### KEEEP

# see which are in the next level
K=2
current_ks=first_ks//K
primal_data = get_pool(global_fine_grid, current_ks)
normals, vstars, _, _ = DiffusionTensor.get_feature_data(primal_data.jdata)

sub_features, sub_grid = occupancy_tensor.grid.subdivide(K, occupancy_tensor.feature)
sub_centers = sub_grid.grid_to_world(sub_grid.ijk.float())

is_void = (primal_data.grid.ijk_to_index(sub_grid.ijk).jdata == -1)[:, None] # subdivided voxel not in grid

to_decide_sign = ((sub_features==0)*(is_void)).jdata # --> Decide sign
# else keep value 

# --> Decide sign
points_neigh_idx = primal_data.grid.neighbor_indexes(sub_grid.ijk, 2).jdata
mask_exists = (points_neigh_idx!=-1)*1.

local_delta = (sub_centers.jdata.unsqueeze(1).unsqueeze(1).unsqueeze(1)-vstars[points_neigh_idx])
local_sdf = (local_delta*normals[points_neigh_idx]).sum(-1)
local_sdf[torch.logical_not(mask_exists)] = 0
local_dist = (local_delta**2).sum(-1)
local_dist[torch.logical_not(mask_exists)] = 1e20

predicted_sdf = (torch.softmax(-10000*local_dist.view(len(is_void), -1), -1)*local_sdf.view(len(is_void), -1))
predicted_sdf = predicted_sdf.sum(-1, True)

# sub_features.jdata[to_decide_sign] = 2.*(predicted_sdf[to_decide_sign]>0)-1
sub_features.jdata[to_decide_sign] = predicted_sdf[to_decide_sign]

# prune
sub_neigh_idx = sub_grid.neighbor_indexes(sub_grid.ijk, 1).jdata

# TODO

occupancy_tensor = fvnn.VDBTensor(sub_grid, sub_features)

first_ks = current_ks

### Occupancy tensor: contains atoccupancy_tensor.grid.subdivide(2, occupancy_tensor.feature) each level, +1 for inside voxels, -1 for outside voxels, 0 for voxel containing the surface

In [12]:
predicted_sdf

tensor([[0.0458],
        [0.0182],
        [0.0346],
        [0.0070],
        [0.0474],
        [0.0072],
        [0.0414],
        [0.0012],
        [0.0206],
        [0.0487],
        [0.0081],
        [0.0362],
        [0.0070],
        [0.0455],
        [0.0014],
        [0.0398],
        [0.0169],
        [0.0043],
        [0.0321],
        [0.0195],
        [0.0070],
        [0.0052],
        [0.0284],
        [0.0266],
        [0.0047],
        [0.0038],
        [0.0243],
        [0.0233],
        [0.0070],
        [0.0023],
        [0.0329],
        [0.0282],
        [0.0483],
        [0.0073],
        [0.0421],
        [0.0011],
        [0.0480],
        [0.0177],
        [0.0370],
        [0.0068],
        [0.0080],
        [0.0504],
        [0.0014],
        [0.0438],
        [0.0182],
        [0.0460],
        [0.0067],
        [0.0345],
        [0.0066],
        [0.0060],
        [0.0285],
        [0.0279],
        [0.0147],
        [0.0051],
        [0.0344],
        [0

In [13]:
# ### KEEEP
# # prune
# sub_neigh_idx = sub_grid.neighbor_indexes(sub_grid.ijk, 1).jdata

# subdivided_occupancy_grid = fvnn.VDBTensor(sub_grid, sub_features)
# # subdivided_occupancy_grid = fvnn.VDBTensor(sub_grid, sub_grid.jagged_like(p_sdf))

In [14]:
mt.plotSlice(primal_data.to_dense()[0, ..., -1].cpu().detach().numpy(), 1.)

interactive(children=(IntSlider(value=2, description='s', max=3), Output()), _dom_classes=('widget-interact',)…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [15]:
mt.plotSlice(occupancy_tensor.to_dense()[0, ..., 0].cpu().detach().numpy(), .1)

interactive(children=(IntSlider(value=2, description='s', max=3), Output()), _dom_classes=('widget-interact',)…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

### Semi-Robust labeling

In [2]:
ind=2
tens = torch.load('../../output/stone-cliff/gen_0_4.pt', weights_only=False)
source_tensor = DiffusionTensor(tens.grid[ind], tens.feature[ind]).remove_mask()

In [3]:
mt.plotSlice(source_tensor.to_dense()[0, ..., 6].cpu().detach().numpy(), 1.)

interactive(children=(IntSlider(value=110, description='s', max=219), Output()), _dom_classes=('widget-interac…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [4]:

mt.mesh_grid(3, True)

array([[-1., -1., -1.],
       [-1., -1.,  0.],
       [-1., -1.,  1.],
       [-1.,  0., -1.],
       [-1.,  0.,  0.],
       [-1.,  0.,  1.],
       [-1.,  1., -1.],
       [-1.,  1.,  0.],
       [-1.,  1.,  1.],
       [ 0., -1., -1.],
       [ 0., -1.,  0.],
       [ 0., -1.,  1.],
       [ 0.,  0., -1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1., -1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  1.],
       [ 1., -1., -1.],
       [ 1., -1.,  0.],
       [ 1., -1.,  1.],
       [ 1.,  0., -1.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  1.],
       [ 1.,  1., -1.],
       [ 1.,  1.,  0.],
       [ 1.,  1.,  1.]])

In [6]:
def create_dilated_grid(grid:fvdb.GridBatch):
    dilated_ijk = grid.ijk.jdata.clone()
    dilated_ijk = dilated_ijk[:, None] + torch.tensor(mt.mesh_grid(3, True), device=dilated_ijk.device)
    dilated_ijk = dilated_ijk.view(-1, 3).long()
    return fvdb.sparse_grid_from_ijk(dilated_ijk, origins=grid.origins, voxel_sizes=grid.voxel_sizes)


def label_dilated_grid(source_tensor: fvnn.VDBTensor, dilated_grid: fvdb.GridBatch):
    '''Label dilated grid based on the normal orientation: 0 (intersects surface), 1 or -1 (resp. strictly outside/inside)'''
    dilated_grid = create_dilated_grid(source_tensor.grid)
    dilated_tensor = grid_to_VDB(dilated_grid, torch.ones, additional_feat=[1])
    dilated_centers = dilated_grid.grid_to_world(dilated_grid.ijk.float())
    neighbors = source_tensor.grid.neighbor_indexes(dilated_tensor.grid.ijk, 1)
    
    # assign existing voxels to 0
    in_source_mask = neighbors.jdata[:, 1, 1, 1]!=-1
    
    # --> Decide sign
    flat_neighbors = neighbors.jdata.view(-1, 27)
    normals, vstars, _, _ = DiffusionTensor.get_feature_data(source_tensor.get_global().jdata)
    local_delta = (dilated_centers.jdata.unsqueeze(1)-vstars[flat_neighbors])
    local_sdf = (local_delta*normals[flat_neighbors]).sum(-1)
    local_sdf[flat_neighbors==-1] = 0
    local_sdf = local_sdf.sum(-1)/(flat_neighbors!=-1).sum(-1)

    dilated_tensor.feature.jdata = local_sdf[:, None]
    dilated_tensor.feature.jdata[in_source_mask] = 0
    
    return dilated_tensor

# def create_dilated_dual(source_tensor: fvnn.VDBTensor, dilated_tensor: fvnn.VDBTensor):
#     dilated_centers = dilated_tensor.grid.grid_to_world(dilated_tensor.grid.ijk.float())
#     dual_grid = dilated_tensor.grid.dual_grid()
#     new_feature = dual_grid.splat_trilinear(dilated_centers, dilated_tensor.feature)
#     dual_tensor = fvdb.nn.VDBTensor(dual_grid, new_feature)

In [7]:
dilated_grid = create_dilated_grid(source_tensor.grid)
dilated_tensor = label_dilated_grid(source_tensor, dilated_grid)

In [8]:
mt.plotSlice(dilated_tensor.to_dense()[0, ..., -1].cpu().detach().numpy(), .001)

interactive(children=(IntSlider(value=111, description='s', max=221), Output()), _dom_classes=('widget-interac…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [9]:
dual_grid = dilated_tensor.grid.dual_grid()
dual_centers = dual_grid.grid_to_world(dual_grid.ijk.float())
dilated_centers = dilated_grid.grid_to_world(dilated_grid.ijk.float())

new_feature = dual_grid.splat_trilinear(dilated_centers, dilated_tensor.feature)



dual_tensor = fvdb.nn.VDBTensor(dual_grid, new_feature)




In [10]:
new_feature

In [11]:
mt.plotSlice(dual_tensor.to_dense()[0, ..., -1].cpu().detach().numpy(), .000001)

interactive(children=(IntSlider(value=111, description='s', max=222), Output()), _dom_classes=('widget-interac…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [12]:
dual_tensor.grid.origins

tensor([[0., 0., 0.]], device='cuda:0')

In [13]:
# consolidate 
dual_neighbors = dual_tensor.grid.neighbor_indexes(dual_tensor.grid.ijk, 1).jdata.view(-1, 27)

neigh_data = dual_tensor.jdata[dual_neighbors].squeeze()
neigh_data[dual_neighbors==-1] = 0
neigh_data = neigh_data.sum(-1)/(dual_neighbors!=-1).sum(-1)

dual_tensor.feature.jdata[dual_tensor.jdata==0] = neigh_data[dual_tensor.jdata.squeeze()==0]

In [14]:
mt.plotSlice(dual_tensor.to_dense()[0, ..., -1].cpu().detach().numpy(), .000001)

interactive(children=(IntSlider(value=111, description='s', max=222), Output()), _dom_classes=('widget-interac…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [15]:
show_vdb_marching_cubes(dual_tensor)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000209…

### Neighbor based dual contouring

In [16]:
def dual_contouring(dual_tensor: fvnn.VDBTensor):
    ''' dual_tensor: sign on corner vertices
        based on dual edges (neighbors in the graph)'''
    dual_neighbors = dual_tensor.grid.neighbor_indexes(dual_tensor.grid.ijk, 1).jdata
    sign_data = dual_tensor.jdata.sign().squeeze()
    if 0 in sign_data:
        print('WARNING, WRONG SIGN DATA')

    dkx = dual_neighbors[:, 2, 1, 1]
    active_dkx = ((sign_data*sign_data[dkx])==-1)*(dkx!=-1)

    dky = dual_neighbors[:, 1, 2, 1]
    active_dky = ((sign_data*sign_data[dky])==-1)*(dky!=-1)

    dkz = dual_neighbors[:, 1, 1, 2]
    active_dkz = ((sign_data*sign_data[dkz])==-1)*(dkz!=-1)
    
    dkx_face_idx = torch.tensor([[0, -1, -1], [0, 0, -1], [0, 0, 0], [0, -1, 0]], device=dkx.device)
    dky_face_idx = torch.tensor([[-1, 0, 0], [0, 0, 0], [0, 0, -1], [-1, 0, -1]], device=dky.device)
    dkz_face_idx = torch.tensor([[-1, -1, 0], [0, -1, 0], [0, 0, 0], [-1, 0, 0]], device=dkz.device)


    faces = []
    for dk_face_idx, active_dk in zip([dkx_face_idx, dky_face_idx, dkz_face_idx], [active_dkx,  active_dky, active_dkz]):
        face_idx = (dk_face_idx[:, None] + dual_tensor.grid.ijk.jdata[active_dk][None, :]).permute((1, 0, 2))
        face_idx[sign_data[active_dk]>0] = face_idx[sign_data[active_dk]>0].flip(1)
        faces.append(face_idx)
    return torch.concatenate(faces)

In [17]:
def faces_ijk_to_index(candidate_faces: torch.tensor, source_grid: fvdb.GridBatch):
    df = source_tensor.grid.ijk_to_index(candidate_faces.view(-1, 3)).jdata.view(-1, 4)
    return df



candidate_faces = dual_contouring(dual_tensor)
faces = faces_ijk_to_index(candidate_faces, source_tensor.grid)
faces = faces[(faces!=-1).all(-1)].cpu().detach().numpy()

_, vstars, _, _ = DiffusionTensor.get_feature_data(source_tensor.get_global().jdata)


In [18]:
plot(vstars.cpu().detach().numpy(), faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.9574842…

In [19]:
mt.export_obj(vstars.cpu().detach().numpy(), faces, 'test.obj')